---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

# Pilotage de filtres vidéo avec les doigts (MediaPipe Hands + OpenCV)

Objectif de cette activité : utiliser la main (1 à 5 doigts levés) devant la webcam pour changer en temps réel un filtre d'image appliqué à la vidéo.

Les filtres disponibles sont :

- `original` : image normale
- `gray` : niveaux de gris
- `blur` : flou gaussien
- `sepia` : effet sépia
- `edges` : détection de contours (Canny)

Le code utilisé dans ce notebook est **exactement** celui fourni (`filters.py`, `gesture_control.py`, `main.py`), sans modification.


## 1. Installation des bibliothèques nécessaires

On utilise :

- `opencv-python` pour accéder à la webcam et appliquer les filtres ;
- `mediapipe` pour détecter la main et compter le nombre de doigts levés.

Exécute la cellule suivante **une seule fois** (ou dans un nouvel environnement) pour installer les bibliothèques.


In [ ]:
# À exécuter si nécessaire (dans un nouvel environnement par exemple)
# Le paramètre --quiet permet d'alléger l'affichage.
!pip install opencv-python mediapipe --quiet

## 2. Repères de la main (21 points MediaPipe)

MediaPipe Hands repère la main à l'aide de 21 points (landmarks) numérotés.

Les indices suivants sont utilisés pour détecter les doigts levés :

- 4 : bout du pouce
- 8 : bout de l'index
- 12 : bout du majeur
- 16 : bout de l'annulaire
- 20 : bout de l'auriculaire

Le script `gesture_control.py` compte combien de doigts sont levés et renvoie une chaîne de la forme `"3_fingers"`, `"4_fingers"`, etc.


## 3. Création du fichier `filters.py`

Cette cellule crée le fichier `filters.py` contenant les filtres d'image.  
**Ne pas modifier le code ci-dessous.**

In [ ]:
%%writefile filters.py
import cv2
import numpy as np

def apply_filter(frame, filter_name):

    if filter_name == "gray":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    elif filter_name == "sepia":
        sepia_filter = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        return cv2.transform(frame, sepia_filter)
    elif filter_name == "blur":
        return cv2.GaussianBlur(frame, (15,15),0)
    elif filter_name == "edges":
        return cv2.Canny(frame,100,200)

    return frame

## 4. Création du fichier `gesture_control.py`

Cette cellule crée le fichier `gesture_control.py` qui :

1. Utilise MediaPipe pour détecter la main ;
2. Récupère les 21 points caractéristiques ;
3. Compte le nombre de doigts levés ;
4. Renvoie un texte du type `"1_fingers"`, `"2_fingers"`, etc.

**Ne pas modifier le code ci-dessous.**

In [ ]:
%%writefile gesture_control.py
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

def detect_gesture(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            lm_list= []
            for id, lm in enumerate(hand_landmarks.landmark):
                h,w,c = frame.shape
                lm_list.append((int(lm.x *w), int(lm.y * h)))

            fingers_up = 0
            tip_ids = [4,8,12,16,20]

            if lm_list[tip_ids[0]][0] > lm_list[tip_ids[0]-1][0]:
                fingers_up += 1

            for i in range(1,5):
                if lm_list[tip_ids[i]][1] < lm_list[tip_ids[i]-2][1]:
                    fingers_up += 1

            return f"{fingers_up}_fingers"
    return None


## 5. Création du fichier `main.py`

Ce script :

1. Ouvre la webcam ;
2. Utilise `detect_gesture` pour connaître le nombre de doigts levés ;
3. Choisit le filtre correspondant :
   - 1 doigt : `original`
   - 2 doigts : `gray`
   - 3 doigts : `blur`
   - 4 doigts : `sepia`
   - 5 doigts : `edges`
4. Affiche le flux vidéo avec le filtre appliqué et le nom du filtre.

**Le code est recopié tel quel, sans modification.**

In [ ]:
%%writefile main.py
import cv2
from filters import apply_filter
from gesture_control import detect_gesture


cap = cv2.VideoCapture(0)
current_filter = "original"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gesture = detect_gesture(frame)

    if gesture == "1_fingers":
        current_filter = "original"
    if gesture == "2_fingers":
        current_filter = "gray"
    if gesture == "3_fingers":
        current_filter = "blur"
    if gesture == "4_fingers":
        current_filter = "sepia"
    if gesture == "5_fingers":
        current_filter = "edges"

    filtered = apply_filter(frame, current_filter)
    if len(filtered.shape)== 2:
        filtered = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)

    cv2.putText(filtered, f"Current Filter:{current_filter}",(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow("Gesture Controlled Filters", filtered)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

## 6. Lancer le programme de pilotage par les doigts

Exécute la cellule suivante pour lancer le programme `main.py`.

- Une fenêtre intitulée `Gesture Controlled Filters` doit s'ouvrir ;
- Place ta main devant la webcam ;
- Lève 1, 2, 3, 4 ou 5 doigts pour changer de filtre ;
- Appuie sur la touche `ESC` (Échap) pour fermer la fenêtre et arrêter le programme.

⚠️ Si ta machine n'a pas de webcam ou si `cv2.VideoCapture(0)` ne fonctionne pas, tu devras adapter l'index de la caméra (1, 2, etc.) directement dans `main.py`.


In [ ]:
# Lancement du programme principal
!python main.py

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---